In [ ]:
# Necessary import
import pickle
import pandas as pd

# Python version
!python -V

In [ ]:
# Check sci-kit learn version
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [ ]:
# Open the model file
with open('model.bin', 'rb') as f_in:
    # Load the encoder and the model
    dv, model = pickle.load(f_in)

In [ ]:
# Set of categrical features
categorical = ['PULocationID', 'DOLocationID']

# Function for reading the data
def read_data(filename):
    # Read the parquet file
    df = pd.read_parquet(filename)
    
    # # Feature engineering for creating the duration column
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # Convert durations into minutes
    df.duration = df.duration.dt.total_seconds() / 60

    # Filtering for trips lasting from 1 min to 1 hour
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # Fill missing values and conert data type
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    # Return the dataframe
    return df

In [ ]:
# Set the date
year, month = 2023, 1

# Read the Yellow taxi data
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [ ]:
# Get the validation data dictionaries
dicts = df[categorical].to_dict(orient = 'records')
# Encode validation data
X_val = dv.transform(dicts)
# Make predictions
y_pred = model.predict(X_val)

---